# Digital acceptance testing

Goal is to make it easy to modify f-eng configuration while recording data.

1. Write MS file
- set all inputs to zeros
- set pol a inputs to adc, then pol b to adc
- set eq coeffs to zero for first ten channels on first snap, then iterate through all snaps

2. Write beamformed data
- set inputs to zeros
- set pol a inputs to adc, then pol b to adc
- set eq coeffs to zero for first ten channels on first snap, then iterate through all snaps
3. Read autos from MS file
- confirm order of zeros, adc on pol on sequence, channel off sequence, and antennas off sequence
4. Read beamformed data
- confirm order of zeros, adc on pol order, channel off order by channel

### Setup

In [22]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from lwa_f import snap2_fengine, snap2_feng_etcd_client
from lwa_antpos import mapping
import casacore.tables

In [8]:
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Viridis
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

output_notebook()

Loading BokehJS ...

In [9]:
snap2names = [f'snap{i:02}' for i in range(1, 12)]

def set_input(mode='use_zero', stream=None, pol=None, names=snap2names):
    """ Set the input block method
    mode can be "use_zero", "use_adc", "use_noise", "zero_coeffs",
    "reset_coeffs", "get_status".
    """

    if isinstance(names, str):
        names = [names]

    for name in names:
        f = snap2_fengine.Snap2Fengine(name)
        num = int(name.lstrip('snap'))

        if (pol is None) or isinstance(stream, int):
            print(f"Set stream: {stream}")
            run(f, stream, mode=mode)
        elif (stream is None) and (pol is not None):
            antpols = []
            if pol == 'a':
                inps = list(range(0,64,2))
            elif pol == 'b':
                inps = list(range(1,64,2))
            for inp in inps:
                antpols.append(mapping.snap2_to_antpol(num, inp))
                run(f, inp, mode=mode)
            print(f"Set antpols {antpols}")

def run(f, stream, mode='use_zero', zerochans=range(0, 5)):
    if mode == 'use_zero':
        f.input.use_zero(stream)
    elif mode == 'use_adc':
        f.input.use_adc(stream)
    elif mode == 'use_noise':
        f.input.use_noise(stream)
    elif mode == 'get_status':
        print('input status:')
        f.input.get_status()
        print('eqtvg status:')
        f.eqtvg.get_status()
    elif mode == 'reset_coeffs':
        f.eq.initialize(read_only=False)
    elif mode == 'zero_coeffs':
        for i in range(64):
            coeffs, scalefactor = f.eq.get_coeffs(i)
            # [1] is scale factor (coeff binary point), coeff = 9600/2**scalefactor?
            coeffs2 = coeffs >> zeropoint
            coeffs2[:5] = 0
            f.eq.set_coeffs(i, coeffs=coeffs2)
    else:
        print(f'mode {mode} not recognized')
# or
#        f.eqtvg.tvg_enable()
#        f.eqtvg.write_stream_tvg()
#        f.eqtvg.write_const_per_stream()
#        f.eqtvg.write_freq_ramp()

def plot_autospecs(names=snap2names, ret=False, bokeh=True):
    """ plots auto correlations for sets of snap2names.
    Can optionally return array with ret=True.
    """

    if isinstance(names, str):
        names = [names]

    autospecs = []
    for name in names:
        f = snap2_fengine.Snap2Fengine(name)
        for i in range(64):
            spec = f.corr.get_new_corr(i, i).real  # 8-channel average, normalized by accumulated time/chans
            autospecs.append(spec)
    autospecs = np.vstack(autospecs)

    if bokeh:
        TOOLTIPS = [("x", "$x"), ("y", "$y"), ("value", "@image")]
        plot = figure(plot_width=800, plot_height=400, tooltips=TOOLTIPS)
        _ = plot.image(image=[autospecs], x=0, y=0, dw=512, dh=len(autospecs), palette=Viridis[4])
        plot.xaxis.axis_label = 'Channel (8-chan avg)'
        plot.yaxis.axis_label = 'Input (64 per snap, stacked)'
        show(plot)
    else:
        fig, (ax0) = plt.subplots(1, 1, figsize=(12,12))
        ax0.imshow(autospecs, origin='lower', interpolation='nearest')

    if ret:
        return autospecs

In [10]:
#f.program('/home/ubuntu/proj/lwa-shell/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg')

In [11]:
# optionally run cold start

cold_start = False
program = False

if cold_start or program:
    for snap2name in snap2names:
        f = snap2_fengine.Snap2Fengine(snap2name)
        if program:
            f.program('/home/ubuntu/proj/lwa-shell/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-01-10_2153.fpg')
        print(snap2name, f.fpga.is_programmed())
        f.cold_start_from_config('/home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml') 

### Visualize initial state

In [12]:
set_input(mode='use_adc')

2022-05-17 22:17:54,507 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:17:54,734 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:17:55,466 - lwa_f.blocks.block:snap02 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:17:55,708 - lwa_f.blocks.block:snap02 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:17:56,448 - lwa_f.blocks.block:snap03 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:17:59,783 - lwa_f.blocks.block:snap03 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:18:00,478 - lwa_f.blocks.block:snap04 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:18:00,704 - lwa_f.blocks.block:snap04 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:18:01,413 - lwa_f.blocks.block:snap05 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:18:01,639 - lwa_f.

In [13]:
autospecs = plot_autospecs(ret=True)

2022-05-17 22:18:11,289 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:18:11,514 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2022-05-17 22:18:25,849 - lwa_f.blocks.block:snap02 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:18:26,106 - lwa_f.blocks.block:snap02 - INFO - adc - Detected FMC ADC board on port 1
2022-05-17 22:18:40,370 - lwa_f.blocks.block:snap03 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:18:40,622 - lwa_f.blocks.block:snap03 - INFO - adc - Detected FMC ADC board on port 1
2022-05-17 22:18:55,145 - lwa_f.blocks.block:snap04 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:18:55,382 - lwa_f.blocks.block:snap04 - INFO - adc - Detected FMC ADC board on port 1
2022-05-17 22:19:09,814 - lwa_f.blocks.block:snap05 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:19:10,063 - lwa_f.blocks.block:snap05 - INFO - adc - Detected FMC ADC board on port 1


### Observing sequence


In [14]:
from lwa352_pipeline_control import Lwa352CorrelatorControl 
p = Lwa352CorrelatorControl( ['lxdlwagpu0%d' % i for i in range(1,9) ], npipeline_per_host=4)

In [15]:
p.pipelines_are_up(verbose=True) 

lxdlwagpu01:0 - up? True
lxdlwagpu01:1 - up? True
lxdlwagpu01:2 - up? True
lxdlwagpu01:3 - up? True
lxdlwagpu02:0 - up? False
lxdlwagpu02:1 - up? False
lxdlwagpu02:2 - up? False
lxdlwagpu02:3 - up? False
lxdlwagpu03:0 - up? False
lxdlwagpu03:1 - up? False
lxdlwagpu03:2 - up? False
lxdlwagpu03:3 - up? False
lxdlwagpu04:0 - up? False
lxdlwagpu04:1 - up? False
lxdlwagpu04:2 - up? False
lxdlwagpu04:3 - up? False
lxdlwagpu05:0 - up? False
lxdlwagpu05:1 - up? False
lxdlwagpu05:2 - up? False
lxdlwagpu05:3 - up? False
lxdlwagpu06:0 - up? False
lxdlwagpu06:1 - up? False
lxdlwagpu06:2 - up? False
lxdlwagpu06:3 - up? False
lxdlwagpu07:0 - up? False
lxdlwagpu07:1 - up? False
lxdlwagpu07:2 - up? False
lxdlwagpu07:3 - up? False
lxdlwagpu08:0 - up? False
lxdlwagpu08:1 - up? False
lxdlwagpu08:2 - up? False
lxdlwagpu08:3 - up? False


False

In [16]:
from mnc.ezdr import Lwa352RecorderControl 
rs = Lwa352RecorderControl('slow')


Found 16 slow recorders: drvs2501 drvs2502 drvs2503 drvs2504 drvs2505 drvs2506 drvs2507 drvs2508 drvs7301 drvs7302 drvs7303 drvs7304 drvs7305 drvs7306 drvs7307 drvs7308


In [21]:
tblock = 3*10  # seconds

set_input(mode='use_zero')
time.sleep(tblock)
set_input(mode='use_adc', pol='a')
time.sleep(tblock)
set_input(mode='use_adc', pol='b')
time.sleep(tblock)
#for name in snap2names:
#    set_input(mode='zero_coeffs', names=name) # zeros some channels
#    time.sleep(tblock)
set_input(mode='reset_coeffs')
time.sleep(tblock)
set_input(mode='use_zero')
time.sleep(tblock)
set_input(mode='use_adc')
time.sleep(tblock)

2022-05-17 22:29:01,268 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:29:01,519 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:29:02,234 - lwa_f.blocks.block:snap02 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:29:02,460 - lwa_f.blocks.block:snap02 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:29:03,155 - lwa_f.blocks.block:snap03 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:29:03,393 - lwa_f.blocks.block:snap03 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:29:04,084 - lwa_f.blocks.block:snap04 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:29:04,294 - lwa_f.blocks.block:snap04 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:29:05,014 - lwa_f.blocks.block:snap05 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:29:05,229 - lwa_f.

2022-05-17 22:29:50,753 - lwa_f.blocks.block:snap11 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:29:50,976 - lwa_f.blocks.block:snap11 - INFO - adc - Detected FMC ADC board on port 1
Set antpols ['LWA-177A', 'LWA-247A', 'LWA-180A', 'LWA-179A', 'LWA-183A', 'LWA-181A', 'LWA-212A', 'LWA-211A', 'LWA-214A', 'LWA-213A', 'LWA-243A', 'LWA-215A', 'LWA-218A', 'LWA-217A', 'LWA-221A', 'LWA-219A', 'LWA-241A', 'LWA-223A', 'LWA-244A', 'LWA-242A', 'LWA-246A', 'LWA-245A', 'LWA-249A', 'LWA-248A', 'LWA-251A', 'LWA-250A', 'LWA-261A', 'LWA-256A', 'LWA-300A', 'LWA-264A', 'LWA-315A', 'LWA-304A']
2022-05-17 22:30:21,653 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:30:21,870 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
Set antpols ['LWA-266B', 'LWA-259B', 'LWA-268B', 'LWA-267B', 'LWA-271B', 'LWA-269B', 'LWA-276B', 'LWA-273B', 'LWA-278B', 'LWA-277B', 'LWA-282B', 'LWA-281B', 'LWA-307B', 'LWA-285B', 'LWA-309B', 'LWA-308B'

Set stream: None
2022-05-17 22:31:14,355 - lwa_f.blocks.block:snap06 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:31:14,565 - lwa_f.blocks.block:snap06 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:31:15,471 - lwa_f.blocks.block:snap07 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:31:16,834 - lwa_f.blocks.block:snap07 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:31:17,832 - lwa_f.blocks.block:snap08 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:31:18,083 - lwa_f.blocks.block:snap08 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:31:18,975 - lwa_f.blocks.block:snap09 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:31:19,220 - lwa_f.blocks.block:snap09 - INFO - adc - Detected FMC ADC board on port 1
Set stream: None
2022-05-17 22:31:20,099 - lwa_f.blocks.block:snap10 - INFO - adc - Detected FMC ADC board on port 0
2022-05-17 22:3

In [5]:
from mnc.ezdr import Lwa352RecorderControl 
rs = Lwa352RecorderControl('slow')
rs.stop()

Found 0 slow recorders: 


[]

In [6]:
from lwa352_pipeline_control import Lwa352CorrelatorControl 
p = Lwa352CorrelatorControl( ['lxdlwagpu0%d' % i for i in range(1,9) ], npipeline_per_host=4)  
p.stop_pipelines()

### Inspect visibility data (need to copy it or run from lxdlwagpu09)

In [18]:
def read_mstar(filename):                                                         
    print(f"reading from {filename}")                                                                                                 
    assert filename.endswith('.tar')
    os.system(f"tar xvf {filename}")
    tb = casacore.tables.table(filename.rstrip('.tar'))
    os.system(f"tar cvf {filename} {filename.rstrip('.tar')}")
    return tb

def plot_data(tb):
    tb2 = tb.query('ANTENNA1=ANTENNA2')
    data = tb2.getcol("DATA").real
    times = tb2.getcol("TIME")
    print(data.shape, times.shape)
    fig, ax = plt.subplots(1, 1, figsize=(12,5))
    ax.imshow(data[..., 0].transpose(), origin='lower', interpolation='nearest')
    ax.set_xlabel('antenna')
    ax.set_ylabel('channel')

In [25]:
glob.glob("/home/ubuntu/data/slow/20220517*15MHz.ms.tar")

[]

In [89]:
tb = read_mstar("/home/ubuntu/data/slow/20220517_222828_15MHz.ms.tar")
#tb = casacore.tables.table('/home/ubuntu/data/slow/20220310_020859_84MHz.ms')

Successful readonly open of default-locked table /home/ubuntu/data/slow/20220310_020859_84MHz.ms: 22 columns, 62128 rows


In [ ]:
plot_data(tb)